Semantic Similarity is the task of determining how similar two sentences are, in terms of what they mean. This example demonstrates the use of SNLI (Stanford Natural Language Inference) Corpus to predict sentence semantic similarity with Transformers. We will fine-tune a BERT model that takes two sentences as inputs and that outputs a similarity score for these two sentences.

Reference:
https://nlp.stanford.edu/projects/snli/

## Setup

#### Setting up logging level
`os.environ['TF_CPP_MIN_LOG_LEVEL']` 
The  `TF_CPP_MIN_LOG_LEVEL` environment variable controls the logging output of TensorFlow, including the debugging information you mentioned.

Setting `TF_CPP_MIN_LOG_LEVEL` to different values will affect the level of logging messages printed to the console:

```
# Do not log any messages
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

# Log only error messages
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

# Log error and warning messages (default behavior)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 

# Log all messages
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0' 
```

You can set this environment variable before running your TensorFlow code to control the logging output.

Note that disabling all logging can make debugging more difficult, so it is generally recommended to set TF_CPP_MIN_LOG_LEVEL to at least 1 to see warning messages.

#### Importing the required libraries

`import numpy as np`

NumPy is a Python library used for working with arrays , which serves as the building block for most numerical computation in Python.

https://numpy.org

`import pandas as pd`

The Pandas library is a Python package designed for data manipulation and analysis.

https://pandas.pydata.org/

`import tensorflow as tf`

The TensorFlow library is a popular open-source software library for building and training machine learning models. It was developed by the Google Brain team and is widely used in many applications, such as image and speech recognition, natural language processing, and recommendation systems.

https://www.tensorflow.org/

`import transformers`

The transformers library is a popular open-source library developed by Hugging Face for natural language processing (NLP) tasks like text classification , information extraction, and question answering. It provides a wide range of pre-trained models for various NLP tasks, including state-of-the-art transformer-based models like BERT, GPT-2, and RoBERTa.

https://huggingface.co/docs/transformers/installation

In [17]:
# to hide the warnings making my notebook dirty
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import numpy as np
import pandas as pd
import tensorflow as tf
import transformers

## Configuration


In [2]:
max_length = 128  # Maximum length of input sentence to the model.
batch_size = 32
epochs = 2

## Loading the dataset

### About the dataset
The Stanford Natural Language Inference (SNLI) Corpus is a large dataset for natural language understanding tasks that deals with "recognizing textual entailment" (RTE), which is the problem of determining the inference relationship between two sentences . The dataset consists of 570k pairs of human-crafted English sentence-pairs that are manually labeled as entailment, contradiction, or neutral. It is designed to evaluate natural language inference (NLI) systems , which aim to determine if a hypothesis can be inferred from a given premise.

The primary application of the SNLI Corpus is for the training and evaluation of machine learning models on natural language understanding tasks. The dataset has become a standard benchmark for evaluating models in the NLI domain, and many recent state-of-the-art models for NLI have been trained on this dataset.

Overall, the SNLI Corpus has been a significant contribution to the field of natural language understanding, providing researchers with a standard dataset for evaluating and benchmarking NLI systems.

#### Acknowledgements
[Modeling Semantic Containment and Exclusion in Natural Language Inference](https://aclanthology.org/C08-1066) (MacCartney & Manning, COLING 2008)

In [3]:
# Labels in our dataset.
labels = ["contradiction", "entailment", "neutral"]

Meaning of "similarity" label values in our dataset:

- Contradiction: The sentences share no similarity.
- Entailment: The sentences have similar meaning.
- Neutral: The sentences are neutral.

In [4]:
!curl -LO https://raw.githubusercontent.com/MohamadMerchant/SNLI/master/data.tar.gz
!tar -xvzf data.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11.1M  100 11.1M    0     0  1907k      0  0:00:06  0:00:06 --:--:-- 2389k
SNLI_Corpus/
SNLI_Corpus/snli_1.0_dev.csv
SNLI_Corpus/snli_1.0_train.csv
SNLI_Corpus/snli_1.0_test.csv


In [5]:
# There are more than 550k samples in total; we will use 100k for this example.
train_df = pd.read_csv("SNLI_Corpus/snli_1.0_train.csv", nrows=100000)
valid_df = pd.read_csv("SNLI_Corpus/snli_1.0_dev.csv")
test_df = pd.read_csv("SNLI_Corpus/snli_1.0_test.csv")

# Shape of the data
print(f"Total train samples : {train_df.shape[0]}")
print(f"Total validation samples: {valid_df.shape[0]}")
print(f"Total test samples: {valid_df.shape[0]}")

Total train samples : 100000
Total validation samples: 10000
Total test samples: 10000


### Sample Dataset values

```
Sentence1:A person on a horse jumps over a broken down airplane.
Sentence2:A person is training his horse for a competition.
Similarity:neutral

Sentence1:A person on a horse jumps over a broken down airplane.
Sentence2:A person is at a diner, ordering an omelette.
Similarity:contradiction

Sentence1:A person on a horse jumps over a broken down airplane.
Sentence2:A person is outdoors, on a horse.
Similarity:entailment

Sentence1:Children smiling and waving at camera
Sentence2:They are smiling at their parents
Similarity:neutral

Sentence1:Children smiling and waving at camera
Sentence2:There are children present
Similarity:entailment

```

In [6]:
train_df

,similarity,sentence1,sentence2
0,neutral,A person on a horse jumps over a broken down a...,A person is training his horse for a competition.
1,contradiction,A person on a horse jumps over a broken down a...,"A person is at a diner, ordering an omelette."
2,entailment,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse."
3,neutral,Children smiling and waving at camera,They are smiling at their parents
4,entailment,Children smiling and waving at camera,There are children present
...,...,...,...
99995,entailment,People with costumes are gathered in a wooded ...,people wear costumes
99996,contradiction,A girl with a black dress and big white bow st...,a man wearing high heels
99997,entailment,A girl with a black dress and big white bow st...,a girl standing
99998,neutral,A girl with a black dress and big white bow st...,a girl getting ready for photo shoot


### Preprocessing
- Remove any NaN values in the dataset.
- Remove records where the label is not provided and is being set to '-'
- One Hot encoding of labels

In [7]:
print("Number of missing values")
print(train_df.isnull().sum())
train_df.dropna(axis=0, inplace=True)

Number of missing values
similarity    0
sentence1     0
sentence2     3
dtype: int64


In [8]:
print("Training data: Number of labels with '-' values")
print(train_df[train_df.similarity == "-"].count())
print()
print("Validation data: Number of labels with '-' values")
print(valid_df[valid_df.similarity == "-"].count())
print()
print("removing '-' labelled records...")
train_df = (
    train_df[train_df.similarity != "-"]
    .sample(frac=1.0, random_state=0)
    .reset_index(drop=True)
)
valid_df = (
    valid_df[valid_df.similarity != "-"]
    .sample(frac=1.0, random_state=0)
    .reset_index(drop=True)
)

Training data: Number of labels with '-' values
similarity    110
sentence1     110
sentence2     110
dtype: int64

Validation data: Number of labels with '-' values
similarity    158
sentence1     158
sentence2     158
dtype: int64

removing '-' labelled records...


### One-hot encode training, validation, and test labels.

In machine learning, one-hot encoding is a method of representing categorical data in a numerical format that can be easily processed by machine learning algorithms. Each categorical value is represented as a binary vector with all zeros and a single one. The position of the one indicates the category that the value represents.

For example, in our dataset we have three different labels with values "contradiction", "entailment", "neutral".  We can use one-hot encoding to represent this variable as:
```
"contradiction" = [1, 0, 0]

"entailment" = [0, 1, 0]

"neutral" = [0, 0, 1]
```
This representation enables machine learning algorithms to easily process the categorical data, instead of trying to interpret string values or ordinal numeric values.

Overall, one-hot encoding is a common method for preprocessing categorical data in machine learning and is used to convert such variables into a format that can be easily consumed by machine learning algorithms.

In [9]:
def convert_to_number(label):
    if label == 'contradiction':
        return 0
    elif label == 'entailment':
        return 1
    return 2

train_df["label"] = train_df["similarity"].apply(convert_to_number)
y_train = tf.keras.utils.to_categorical(train_df.label, num_classes=3)

valid_df["label"] = valid_df["similarity"].apply(convert_to_number)
y_val = tf.keras.utils.to_categorical(valid_df.label, num_classes=3)

test_df["label"] = test_df["similarity"].apply(convert_to_number)
y_test = tf.keras.utils.to_categorical(test_df.label, num_classes=3)

print("Similarity labels after one-hot encoding")
unique_values = y_train
print(unique_values)

Similarity labels after one-hot encoding
[[1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


Now that we have encoded the labels, we now need to tokenize and encode our sentences for which we would create a data generator which will allow us to do the encoding during the training.

We will extend the keras Sequence class and define our own data generator.

Here's an example usage of the Sequence class:

```
from keras.utils import Sequence

class CustomSequence(Sequence):
    def __init__(self, data, batch_size):
        self.data = data
        self.batch_size = batch_size
    
    def __len__(self):
        return len(self.data) // self.batch_size
    
    def __getitem__(self, idx):
        batch_data = self.data[idx*self.batch_size:(idx+1)*self.batch_size]
        # process batch_data here
        return batch_data
        
```

In this example, we create a custom sequence that inherits from the `Sequence` base class. We define the `__init__()` method to initialize the sequence with some data and a batch size, and the `__len__()` and `__getitem__()` methods to define the length of the sequence and how to get a particular batch of data, respectively. This `CustomSequence` class can then be used as a generator with Keras models.

In [10]:
class BertSemanticDataGenerator(tf.keras.utils.Sequence):
    """Generates batches of data.

    Args:
        sentence_pairs: Array of premise and hypothesis input sentences.
        labels: Array of labels.
        batch_size: Integer batch size.
        shuffle: boolean, whether to shuffle the data.
        include_targets: boolean, whether to incude the labels.

    Returns:
        Tuples `([input_ids, attention_mask, `token_type_ids], labels)`
        (or just `[input_ids, attention_mask, `token_type_ids]`
         if `include_targets=False`)
    """

    def __init__(
        self,
        sentence_pairs,
        labels,
        batch_size=batch_size,
        shuffle=True,
        include_targets=True,
    ):
        self.sentence_pairs = sentence_pairs
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
        # Load our BERT Tokenizer to encode the text.
        # We will use base-base-uncased pretrained model.
        self.tokenizer = transformers.BertTokenizer.from_pretrained(
            "bert-base-uncased", do_lower_case=True
        )
        self.indexes = np.arange(len(self.sentence_pairs))
        self.on_epoch_end()

    def __len__(self):
        # Denotes the number of batches per epoch.
        return len(self.sentence_pairs) // self.batch_size

    def __getitem__(self, idx):
        # Retrieves the batch of index.
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentence_pairs = self.sentence_pairs[indexes]

        # With BERT tokenizer's batch_encode_plus batch of both the sentences are
        # encoded together and separated by [SEP] token.
        encoded = self.tokenizer.batch_encode_plus(
            sentence_pairs.tolist(),
            add_special_tokens=True,
            max_length=max_length,
            return_attention_mask=True,
            return_token_type_ids=True,
            pad_to_max_length=True,
            return_tensors="tf",
        )

        # Convert batch of encoded features to numpy array.
        input_ids = np.array(encoded["input_ids"], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

        # Set to true if data generator is used for training/validation.
        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="int32")
            return [input_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids, attention_masks, token_type_ids]

    def on_epoch_end(self):
        # Shuffle indexes after each epoch if shuffle is set to True.
        if self.shuffle:
            np.random.RandomState(42).shuffle(self.indexes)

### Build a model
We will build our model using the TensorFlow distribution strategy. 

`tf.distribute.MirroredStrategy` is a TensorFlow distribution strategy that is used when training a model on multiple GPUs or a single multi-GPU machine. With this strategy, the model will be replicated across all the available GPUs, and each replica will process a different portion of the input data in parallel. Gradient updates are then combined across the replicas to update the model weights.

When using `MirroredStrategy`, the with `strategy.scope()`: context manager is typically used to define the model and optimizer inside the strategy scope, so that they are automatically distributed across all the replicas. This ensures that the model and optimizer are available on every device in the strategy, and that the variables are automatically mirrored across the devices.

Here's an example usage of `MirroredStrategy` with the scope:

```
import tensorflow as tf

strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    # Define model and optimizer here
    model = ...
    optimizer = ...
    
@tf.function
def train_step(inputs, labels):
    # Perform forward and backward pass here
    with tf.GradientTape() as tape:
        predictions = model(inputs)
        loss = ...
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

```

The `train_step()` function can then be called inside the training loop. When using MirroredStrategy, the batch size should be increased by the number of replicas in order to preserve the same global batch size.

#### BERT tokenizer
A BERT tokenizer, short for Bidirectional Encoder Representations from Transformers tokenizer, is a natural language processing tool used for tokenization, which is the process of breaking up raw text into smaller chunks that can be used for further analysis, such as machine learning or deep learning models.

As opposed to traditional tokenizers that split text into individual words or characters, BERT uses a WordPiece tokenizer, which breaks words down into smaller subword units, such as prefixes or suffixes, to better capture the complex relationships between different words in natural language text. This allows BERT to better handle rare or unknown words, and to generalize better to new text inputs.

#### Attention Masks
In natural language processing (NLP) and specifically in the context of transformers like BERT, an attention mask is an optional input used to specify which tokens of a sequence should be attended to by the model.

In NLP, sequences of words or embeddings are often used as input to models. However, sequences can have varying lengths, so models typically require inputs of fixed length. When batching sequences together for training, sequences in a batch with shorter lengths are padded with zeros to match the length of the longest sequence. The attention mask is used to tell the transformer model which tokens in the input sequence are real tokens and which are just padding tokens.

The attention mask is usually a tensor of 0's and 1's with the same shape as the input sequence. It has a value of 1 for real tokens and 0 for padding tokens. During self-attention mechanism, in which a sequence attends to itself to compute a representation, tokens with value 0 in the mask are ignored so the model does not attend to the padded positions.

Attention masks can also be used to handle input sequences with different lengths by dynamically masking the appropriate tokens, rather than padding all sequences to the same length.

In [11]:
# Create the model under a distribution strategy scope.
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    # Encoded token ids from BERT tokenizer.
    input_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="input_ids"
    )
    # Attention masks indicates to the model which tokens should be attended to.
    attention_masks = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="attention_masks"
    )
    # Token type ids are binary masks identifying different sequences in the model.
    token_type_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="token_type_ids"
    )
    # Loading pretrained BERT model.
    bert_model = transformers.TFBertModel.from_pretrained("bert-base-uncased")
    # Freeze the BERT model to reuse the pretrained features without modifying them.
    bert_model.trainable = False

    bert_output = bert_model.bert(
        input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids
    )
    sequence_output = bert_output.last_hidden_state
    pooled_output = bert_output.pooler_output
    # Add trainable layers on top of frozen layers to adapt the pretrained features on the new data.
    bi_lstm = tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(64, return_sequences=True)
    )(sequence_output)
    # Applying hybrid pooling approach to bi_lstm sequence output.
    avg_pool = tf.keras.layers.GlobalAveragePooling1D()(bi_lstm)
    max_pool = tf.keras.layers.GlobalMaxPooling1D()(bi_lstm)
    concat = tf.keras.layers.concatenate([avg_pool, max_pool])
    dropout = tf.keras.layers.Dropout(0.3)(concat)
    output = tf.keras.layers.Dense(3, activation="softmax")(dropout)
    model = tf.keras.models.Model(
        inputs=[input_ids, attention_masks, token_type_ids], outputs=output
    )

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss="categorical_crossentropy",
        metrics=["acc"],
    )


print(f"Strategy: {strategy}")
model.summary()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Strategy: <tensorflow.python.distribute.mirrored_strategy.MirroredStrategy object at 0x7f8168019fa0>
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 128)]        0           []                               
                                                                                                  
 attention_masks (InputLayer)   [(None, 128)]        0           []                               
                                                                                                  
 token_type_ids (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  109482240   ['input_ids[0][0]',        

### Create training and validation dataset using our custom generator

In [12]:
train_data = BertSemanticDataGenerator(
    train_df[["sentence1", "sentence2"]].values.astype("str"),
    y_train,
    batch_size=batch_size,
    shuffle=True,
)
valid_data = BertSemanticDataGenerator(
    valid_df[["sentence1", "sentence2"]].values.astype("str"),
    y_val,
    batch_size=batch_size,
    shuffle=False,
)

### Train the Model
Training is done only for the top layers to perform "feature extraction", which will allow the model to use the representations of the pretrained model.



In [13]:
history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=epochs,
    use_multiprocessing=True,
    workers=-1,
)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
2023-05-03 14:14:46.127460: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:786] AUTO shard

Epoch 1/2
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tenso

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


  94/3121 [..............................] - ETA: 10:32 - loss: 1.0670 - acc: 0.4461

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


...

3074/3121 [============================>.] - ETA: 9s - loss: 0.5928 - acc: 0.7583

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


3121/3121 [==============================] - ETA: 0s - loss: 0.5926 - acc: 0.7585

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


3121/3121 [==============================] - 683s 219ms/step - loss: 0.5926 - acc: 0.7585 - val_loss: 0.5017 - val_acc: 0.8029


In [15]:
test_data = BertSemanticDataGenerator(
    test_df[["sentence1", "sentence2"]].values.astype("str"),
    y_test,
    batch_size=batch_size,
    shuffle=False,
)
model.evaluate(test_data, verbose=1)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
2023-05-03 14:51:04.135224: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:786] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorDataset/_1"
op: "TensorDataset"
input: "Placeholder/_0"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_INT32
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 1
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\020TensorDataset:47"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
      }
    }
  }
}
experi

 70/312 [=====>........................] - ETA: 43s - loss: 0.5383 - acc: 0.7844

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


 83/312 [======>.......................] - ETA: 40s - loss: 0.5348 - acc: 0.7869

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


312/312 [==============================] - 56s 179ms/step - loss: 0.5236 - acc: 0.7953


[0.5235779881477356, 0.7952724099159241]

In [22]:
def check_similarity(sentence1, sentence2):
    sentence_pairs = np.array([[str(sentence1), str(sentence2)]])
    test_data = BertSemanticDataGenerator(
        sentence_pairs, labels=None, batch_size=1, shuffle=False, include_targets=False,
    )

    proba = model.predict(test_data[0])[0]
    idx = np.argmax(proba)
    return labels[idx], proba[idx]

sentence1 = "Two women are observing something together."
sentence2 = "Two women are standing with their eyes closed."
check_similarity(sentence1, sentence2)[1]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 0s 65ms/step


0.5636335

### News article grouping based on their semantic similarity
We will take a large group of news titles and group them based on their semantic similarity through our model.
```
new_articles = [
    "Brokerages hold mixed views on Adani-owned Ambuja Cements post Q4 results; check details",
    "Adani Wilmar posts 60 pc decline in Q4 profit",
    "Dharavi wants to be redeveloped but with its residents' future secured",
    "Adani Wilmar standalone Q4 net down 65% at ₹98 cr",
    "Adani Wilmar Q4 profit slumps 60% amid fall in edible oil prices",
    "Adani-Hindenburg row: PIL petitioner moves SC, opposes SEBI’s plea for extension of time to complete probe",
    "Adani Group shares weak; Adani Wilmar sheds 5% as Q4FY23 PAT sinks 56% YoY",
    "Adani Wilmar Q4 profit down 60% to ₹94 crore",
    "Hope Sebi will get clarity on foreign funds invested in Adani group: Cong",
    "India’s Adani Wilmar posts 60% drop in Q4 profit on weak edible oils demand",
    "Hindenburg targets Icahn Enterprises, claims IEP shares overvalued",
    "Adani Wilmar reports 60% fall in net profit in Q4FY23, revenue up 7%",
    "Adani Wilmar Q4 results: Profit falls to  ₹94 crore, stock slips nearly 3%",
    "Singapore's Vantage Point is highest bidder for bankrupt SKS Power",
    "Hope SEBI will use all means for clarity on ownership of foreign funds invested in Adani group: Congress",
    "Adani Total Gas Consolidated March 2023 Net Sales at Rs 1,114.78 crore, up 10.15% Y-o-Y",
    "From Adani Wilmar to Havells India: Q4 results to watch out for today",
    "Adani group to set up two data centres in AP with ₹21,844 crore investment",
    "Top Headlines: Go First's bankruptcy, extension on Adani row probe and more",
    "Stocks to watch on May 3, 2023",
    "Stocks to Watch: Tata Steel, Airtel, Ambuja Cements, Hindustan Zinc, Pricol",
    "Adani Wilmar, Titan, Godrej Properties, MRF, Havells India, Tata Chemical, among others to announce Q4 results today",
    "Carl Icahn's wealth plunges $10 bn on Hindenburg short-seller report"
]

```

In [25]:
new_articles = [
    "Brokerages hold mixed views on Adani-owned Ambuja Cements post Q4 results; check details",
    "Adani Wilmar posts 60 pc decline in Q4 profit",
    "Dharavi wants to be redeveloped but with its residents' future secured",
    "Adani Wilmar standalone Q4 net down 65% at ₹98 cr",
    "Adani Wilmar Q4 profit slumps 60% amid fall in edible oil prices",
    "Adani-Hindenburg row: PIL petitioner moves SC, opposes SEBI’s plea for extension of time to complete probe",
    "Adani Group shares weak; Adani Wilmar sheds 5% as Q4FY23 PAT sinks 56% YoY",
    "Adani Wilmar Q4 profit down 60% to ₹94 crore",
    "Hope Sebi will get clarity on foreign funds invested in Adani group: Cong",
    "India’s Adani Wilmar posts 60% drop in Q4 profit on weak edible oils demand",
    "Hindenburg targets Icahn Enterprises, claims IEP shares overvalued",
    "Adani Wilmar reports 60% fall in net profit in Q4FY23, revenue up 7%",
    "Adani Wilmar Q4 results: Profit falls to  ₹94 crore, stock slips nearly 3%",
    "Singapore's Vantage Point is highest bidder for bankrupt SKS Power",
    "Hope SEBI will use all means for clarity on ownership of foreign funds invested in Adani group: Congress",
    "Adani Total Gas Consolidated March 2023 Net Sales at Rs 1,114.78 crore, up 10.15% Y-o-Y",
    "From Adani Wilmar to Havells India: Q4 results to watch out for today",
    "Adani group to set up two data centres in AP with ₹21,844 crore investment",
    "Top Headlines: Go First's bankruptcy, extension on Adani row probe and more",
    "Stocks to watch on May 3, 2023",
    "Stocks to Watch: Tata Steel, Airtel, Ambuja Cements, Hindustan Zinc, Pricol",
    "Adani Wilmar, Titan, Godrej Properties, MRF, Havells India, Tata Chemical, among others to announce Q4 results today",
    "Carl Icahn's wealth plunges $10 bn on Hindenburg short-seller report"
]

# comparing left and right 
similar_articles = []
for current_title_index in np.arange(0, len(new_articles)): # 0-length
    title_left = new_articles[current_title_index]
    for next_title_index in np.arange(current_title_index + 1, len(new_articles)):
        title_right = new_articles[next_title_index]
        similary_percent = check_similarity(title_left, title_right)[1]
        if similary_percent > 0.80: # at least 80% similarity 
            print(title_left)
            print(title_right)
            print(similary_percent)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 0s 63ms/step


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 0s 62ms/step


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


...

1/1 [==============================] - 0s 63ms/step


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 0s 62ms/step


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 0s 65ms/step
Adani Wilmar, Titan, Godrej Properties, MRF, Havells India, Tata Chemical, among others to announce Q4 results today
Carl Icahn's wealth plunges $10 bn on Hindenburg short-seller report
0.8551821
